In [1]:
import logging
log_level = 25
logging.basicConfig(level=log_level) # root logger

import re
import pandas as pd

import sys
sys.path.append('E:/Code/chat/gdpr')


In [1]:
import fitz

import os
path = 'E:/Code/chat/gdpr/pdf/guidelines'
file_name = 'edpb_guidelines_201903_video_devices_en_0.pdf'
full_path = os.path.join(path, file_name)
doc = fitz.open(full_path)

output_file = "../tmp/video.md"

In [5]:
import importlib
import src.extract_from_pdf
importlib.reload(src.extract_from_pdf)
from src.extract_from_pdf import output_doc_as_text

lines_to_delete = [] 
characters_to_replace = []
characters_to_replace.append(['“', '"'])
characters_to_replace.append(['”', '"'])
characters_to_replace.append(['‘', "'"])
characters_to_replace.append(['’', "'"])
characters_to_replace.append(['', "-"])
characters_to_replace.append(['–', "-"])




document_text = output_doc_as_text(doc, start_page = 4, end_page = 0, header_size=70, footer_size=70, lines_to_delete = lines_to_delete, characters_to_replace = characters_to_replace)

with open(output_file, 'w', encoding='utf-8') as f:
    f.write(document_text)

## Once you have the markdown file

In [2]:
import re
import pandas as pd
file_path = "../../original/video.md"
with open(file_path, 'r', encoding = "utf-8") as file:
    text = file.read()

lines = text.split('\n')
# get rid of empty lines
lines = [line for line in lines if line]

doc_as_array = []
notes_as_array = []
#footnote_pattern = re.compile(r'^(\[\^\d{1,2}\]:)(.*)$')
footnote_pattern = re.compile(r'^\[\^(\d{1,2})\]:(.*)$')
for entry in lines:
    footnote_match = footnote_pattern.match(entry)
    if footnote_match:
        notes_as_array.append([footnote_match.group(1), footnote_match.group(2).strip()])
    else:
        doc_as_array.append(entry)

columns = ["note_number", "text"]
df_notes = pd.DataFrame(notes_as_array, columns = columns)
df_notes = df_notes[df_notes["text"].str.strip() != '']

In [3]:
#columns = ["section", "subsection", "point", "subpoint", "heading", "text", "section_reference"]
columns = ["section", "subsection", "point", "heading", "text", "section_reference"]
section = ""
subsection = ""
point = ""
heading = False
text = ""
section_reference = ""

section_pattern = re.compile(r'^(\d+) (?!\.)(.+)$')
subsection_pattern = re.compile(r'(\d\.\d)(?!\.)\s*(.*)') 
point_pattern = re.compile(r'^(\d\.\d\.\d)\s+(.+)$') 
annex_pattern = re.compile(r'(?i)^ANNEX (\d+) - (.+)$') # ignore capitalization


annex_started = False
data = []
table_data = []
for entry in doc_as_array:
    if entry.strip() != '':
        section_match = section_pattern.match(entry)
        subsection_match = subsection_pattern.match(entry)
        point_match = point_pattern.match(entry)
        annex_match = annex_pattern.match(entry)

        if section_match:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                match = section_match
                section = match.group(1)
                subsection = ""
                point = ""
                heading = True
                text = match.group(2)
                section_reference = section
                data.append([section, subsection, point, heading, text, section_reference])
        elif subsection_match:
            match = subsection_match
            section = section
            subsection = match.group(1)
            point = ""
            heading = True
            text = match.group(2)
            section_reference = subsection
            data.append([section, subsection, point, heading, text, section_reference])
        elif point_match:
            match = point_match
            section = section
            subsection = subsection
            point = match.group(1)
            heading = True
            text = match.group(2)
            section_reference = point
            data.append([section, subsection, point, heading, text, section_reference])

        elif annex_match:
            match = annex_match
            annex_started = True
            section = "Annex"
            subsection = ""
            point = ""
            heading = True
            text = match.group(1)
            section_reference = section
            data.append([section, subsection, point, heading, text, section_reference])
        
        else:
            if annex_started:
                data.append(["", "", "", False, entry, "Annex"])
            else:
                section = section
                subsection = subsection
                point = point
                heading = False
                text = entry
                section_reference = section_reference
                
                data.append(["", "", "", heading, text, section_reference])




df = pd.DataFrame(data, columns = columns)
df = df.drop([0,1,2,3,4])
df.reset_index(inplace=True)

#df.loc[df["section_reference"] == "",  "section_reference"] = "INTRODUCTION"
# Remove my note about the table
#df = df[df["text"] != 'Note this table contains a column "References to BCR-C, application form BCR-C, and / or supporting documents[^14]" which is empty in the document because it is supposed to be filled out by the controller'] 

In [4]:
df[220:240]
#df.iloc[192]["text"]
#df[df["text"] == " "]

,index,section,subsection,point,heading,text,section_reference
220,225,,,,False,- Data encryption.,9.3.2
221,226,,,,False,- Use of hardware and software based solutions...,9.3.2
222,227,,,,False,"- Detection of failures of components, softwar...",9.3.2
223,228,,,,False,- Means to restore availability and access to ...,9.3.2
224,229,,,,False,135. Access control ensures that only authoriz...,9.3.2
225,230,,,,False,- Ensuring that all premises where monitoring ...,9.3.2
226,231,,,,False,- Positioning monitors in such a way (especial...,9.3.2
227,232,,,,False,"- Procedures for granting, changing and revoki...",9.3.2
228,233,,,,False,- Methods and means of user authentication and...,9.3.2
229,234,,,,False,- User performed actions (both to the system a...,9.3.2


In [5]:
# Add footnotes
import re

def find_footnote_references(text):
    pattern = r'\[\^(\d+)\]'
    return re.findall(pattern, text)

for index, row in df.iterrows():
    footnotes = find_footnote_references(row['text'])
    if footnotes:
        augmented_note = row['text']
        for note in footnotes:
            augmented_note += f"\n[^{note}]: {df_notes[df_notes['note_number'] == note].iloc[0]['text']}"
        print(f"Row {index} augmented with footnotes")
        #print(augmented_note)
        df.at[index, "text"] = augmented_note
        #print

Row 8 augmented with footnotes
Row 18 augmented with footnotes
Row 19 augmented with footnotes
Row 25 augmented with footnotes
Row 26 augmented with footnotes
Row 33 augmented with footnotes
Row 35 augmented with footnotes
Row 54 augmented with footnotes
Row 57 augmented with footnotes
Row 62 augmented with footnotes
Row 65 augmented with footnotes
Row 68 augmented with footnotes
Row 103 augmented with footnotes
Row 117 augmented with footnotes
Row 160 augmented with footnotes
Row 161 augmented with footnotes
Row 168 augmented with footnotes
Row 182 augmented with footnotes
Row 195 augmented with footnotes
Row 199 augmented with footnotes
Row 200 augmented with footnotes
Row 205 augmented with footnotes
Row 233 augmented with footnotes


In [6]:
file = "../../inputs/documents/video.parquet" # use parquet to deal with the complex text so I don't need to worry about escape characters
df.to_parquet(file, engine = 'pyarrow')

#df_no_table.to_csv(file, encoding = "utf-8", sep="|", index = False, na_rep="", quotechar='"')


## Check that the document class works as expected

In [9]:
import sys
sys.path.append('E:/Code/chat/gdpr')

import importlib
import gdpr_rag.documents.video
importlib.reload(gdpr_rag.documents.video)
from gdpr_rag.documents.video import Video

path_to_manual_as_csv_file = "../../inputs/documents/video.parquet"

doc = Video(path_to_manual_as_csv_file)


In [12]:
from IPython.display import Markdown, display

section = "1"

section = "3.1.1"
#section = "10"

print(doc.get_heading(section))
display(Markdown(doc.get_text(section)))


3 LAWFULNESS OF PROCESSING
3.1 Legitimate interest, Article 6 (1) (f)
3.1.1 Existence of legitimate interests


# 3 LAWFULNESS OF PROCESSING

## 3.1 Legitimate interest, Article 6 (1) (f)

### 3.1.1 Existence of legitimate interests

18. Video surveillance is lawful if it is necessary in order to meet the purpose of a legitimate interest pursued by a controller or a third party, unless such interests are overridden by the data subject's interests or fundamental rights and freedoms (Article 6 (1) (f)). Legitimate interests pursued by a controller or a third party can be legal[^8], economic or non-material interests.[^9] However, the controller should consider that if the data subject objects to the surveillance in accordance with Article 21 the controller can only proceed with the video surveillance of that data subject if it is a compelling legitimate interest which overrides the interests, rights and freedoms of the data subject or for the establishment, exercise or defence of legal claims.

19. Given a real and hazardous situation, the purpose to protect property against burglary, theft or vandalism can constitute a legitimate interest for video surveillance.

20. The legitimate interest needs to be of real existence and has to be a present issue (i.e. it must not be fictional or speculative)[^10]. A real-life situation of distress needs to be at hand - such as damages or serious incidents in the past - before starting the surveillance. In light of the principle of accountability, controllers would be well advised to document relevant incidents (date, manner, financial loss) and related criminal charges. Those documented incidents can be a strong evidence for the existence of a legitimate interest. The existence of a legitimate interest as well as the necessity of the monitoring should be reassessed in periodic intervals (e. g. once a year, depending on the circumstances).

21. Example: A shop owner wants to open a new shop and wants to install a video surveillance system to prevent vandalism. He can show, by presenting statistics, that there is a high expectation of vandalism in the near neighbourhood. Also, experience from neighbouring shops is useful. It is not necessary that a damage to the controller in question must have occurred. As long as damages in the neighbourhood suggest a danger or similar, and thus can be an indication of a legitimate interest. It is however not sufficient to present national or general crime statistic without analysing the area in question or the dangers for this specific shop.

22. Imminent danger situations may constitute a legitimate interest, such as banks or shops selling precious goods (e.g. jewellers), or areas that are known to be typical crime scenes for property offences (e. g. petrol stations).

23. The GDPR also clearly states that public authorities cannot rely their processing on the grounds of legitimate interest, as long as they are carrying out their tasks, Article 6 (1) sentence 2.

  
[^8]: European Court of Justice, Judgment in Case C-13/16, Rīgas satiksme case, 4 may 2017  
[^9]: see WP217, Article 29 Working Party.  
[^10]: see WP217, Article 29 Working Party, p. 24 seq. See also ECJ Case C-708/18 p.44